# Notebook to Train and Experiment with Custom U-Net

### Imports and Global Settings

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn as nn
import torch
from config import config
from utils.data import create_dataloaders
from utils.train import run_training
from utils.evaluation import visualise_batch_predictions, evaluate
from models.unet_model import UNet

In [ ]:
# Set seeds for reproducibility
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
DATA_PATH = config.PATH_TO_DATA
seed = 42
torch.manual_seed(seed)
max_epochs = 10
batch_size = 16
learning_rate = 1e-4
project_name = "U-Net_Baselines"
weight_tensor = torch.tensor([1.0, 1.0], device=device)

In [ ]:
# small functions to fetch and initialise model
def init_weights(m):
  if isinstance(m, nn.Conv2d):
      torch.nn.init.kaiming_normal_(m.weight)

def fetch_UNet(n_channels, n_classes):
  model = UNet(n_channels=n_channels, n_classes=n_classes)
  model.apply(init_weights)
  return model

## R-G-B Baseline

In [ ]:
bands = [0, 1, 2] #R/G/B

train_loader, val_loader, test_loader = create_dataloaders(data_dir=DATA_PATH, batch_size=batch_size, bands=bands)

# define model
model = fetch_UNet(n_channels=len(bands), n_classes=2)
model.to(device)
optimizer = AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=1e-2)

scheduler = CosineAnnealingLR(optimizer, max_epochs)
criterion = torch.nn.CrossEntropyLoss()



In [ ]:
# run training
results = run_training(experiment_name=f"UNet_basemodel_RGB_lr_{learning_rate}_num_epochs_{max_epochs}_lr_{learning_rate}",
                        model=model,
                        num_epochs=max_epochs,
                        optimizer=optimizer,
                        criterion=criterion,
                        train_dl=train_loader,
                        val_dl=val_loader,
                        scheduler=scheduler,
                        lr=learning_rate,
                        batch_size=batch_size,
                        device=device,
                        project_name=project_name
                        )

In [ ]:
#Evaluate test-split performance
result = evaluate(model, device, test_loader, criterion)
print(result)

In [ ]:
#Evaluate validation-split performance
result = evaluate(model, device, val_loader, criterion)
print(result)

#### Visualise some predictions

In [ ]:
loader = iter(test_loader)

In [ ]:
batch_sample, batch_mask = next(loader)
batch_sample, batch_mask = batch_sample.to(device), batch_mask.to(device)
batch_output = model(batch_sample)
batch_predictions = batch_output.argmax(dim=1)
visualise_batch_predictions(batch_sample, batch_mask.unsqueeze(1), batch_predictions.unsqueeze(1), rescale=True, bands=bands)

## Train with R-G-B-NIR

In [ ]:
bands = [0, 1, 2, 3] #R/G/B/NIR

torch.manual_seed(seed)
train_loader, val_loader, test_loader = create_dataloaders(data_dir=DATA_PATH, batch_size=batch_size, bands=bands)

# define model
model = fetch_UNet(n_channels=len(bands), n_classes=2)
model.to(device)
optimizer = AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=1e-2)

scheduler = CosineAnnealingLR(optimizer, max_epochs)
criterion = torch.nn.CrossEntropyLoss(weight=weight_tensor)

In [ ]:
# run training
results = run_training(experiment_name=f"UNet_basemodel_RGBNIR_lr_{learning_rate}_num_epochs_{max_epochs}_lr_{learning_rate}",
                        model=model,
                        num_epochs=max_epochs,
                        optimizer=optimizer,
                        criterion=criterion,
                        train_dl=train_loader,
                        val_dl=val_loader,
                        scheduler=scheduler,
                        lr=learning_rate,
                        batch_size=batch_size,
                        device=device,
                        project_name=project_name
                        )

In [ ]:
#Evaluate test-split performance
result = evaluate(model, device, test_loader, criterion)
print(result)

In [ ]:
#Evaluate validation-split performance
result = evaluate(model, device, val_loader, criterion)
print(result)

#### Visualise some predictions

In [ ]:
loader = iter(test_loader)

In [ ]:
batch_sample, batch_mask = next(loader)
batch_sample, batch_mask = batch_sample.to(device), batch_mask.to(device)
batch_output = model(batch_sample)
batch_predictions = batch_output.argmax(dim=1)
visualise_batch_predictions(batch_sample, batch_mask.unsqueeze(1), batch_predictions.unsqueeze(1), rescale=True, bands=bands)

## Train with R-G-NIR


In [ ]:
bands = [0, 1, 3] #R/G/NIR

torch.manual_seed(seed)
train_loader, val_loader, test_loader = create_dataloaders(data_dir=DATA_PATH, batch_size=batch_size, bands=bands)

# define model
model = fetch_UNet(n_channels=len(bands), n_classes=2)
model.to(device)
optimizer = AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=1e-2)

scheduler = CosineAnnealingLR(optimizer, max_epochs)
criterion = torch.nn.CrossEntropyLoss(weight=weight_tensor)

In [ ]:
# run training
results = run_training(experiment_name=f"UNet_basemodel_RGNIR_lr_{learning_rate}_num_epochs_{max_epochs}_lr_{learning_rate}",
                        model=model,
                        num_epochs=max_epochs,
                        optimizer=optimizer,
                        criterion=criterion,
                        train_dl=train_loader,
                        val_dl=val_loader,
                        scheduler=scheduler,
                        lr=learning_rate,
                        batch_size=batch_size,
                        device=device,
                        project_name=project_name
                        )

In [ ]:
#Evaluate test-split performance
result = evaluate(model, device, test_loader, criterion)
print(result)

In [ ]:
#Evaluate validation-split performance
result = evaluate(model, device, val_loader, criterion)
print(result)

#### Visualise some predictions

In [ ]:
loader = iter(train_loader)

In [ ]:
batch_sample, batch_mask = next(loader)
batch_sample, batch_mask = batch_sample.to(device), batch_mask.to(device)
batch_output = model(batch_sample)
batch_predictions = batch_output.argmax(dim=1)
visualise_batch_predictions(batch_sample, batch_mask.unsqueeze(1), batch_predictions.unsqueeze(1), rescale=True, bands=bands)

### Train with R-G-NDVI

In [ ]:
bands = [0, 1, 4] #R/G/NDVI

torch.manual_seed(seed)
train_loader, val_loader, test_loader = create_dataloaders(data_dir=DATA_PATH, batch_size=batch_size, bands=bands)

# define model
model = fetch_UNet(n_channels=len(bands), n_classes=2)
model.to(device)
optimizer = AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=1e-2)

scheduler = CosineAnnealingLR(optimizer, max_epochs)
criterion = torch.nn.CrossEntropyLoss(weight=weight_tensor)

In [ ]:
# run training
results = run_training(experiment_name=f"UNet_basemodel_RGNDVI_lr_{learning_rate}_num_epochs_{max_epochs}_lr_{learning_rate}",
                        model=model,
                        num_epochs=max_epochs,
                        optimizer=optimizer,
                        criterion=criterion,
                        train_dl=train_loader,
                        val_dl=val_loader,
                        scheduler=scheduler,
                        lr=learning_rate,
                        batch_size=batch_size,
                        device=device,
                        project_name=project_name
                        )

In [ ]:
#Evaluate test-split performance
result = evaluate(model, device, test_loader, criterion)
print(result)

In [ ]:
#Evaluate validation-split performance
result = evaluate(model, device, val_loader, criterion)
print(result)

#### Visualise some predictions

In [ ]:
loader = iter(train_loader)

In [ ]:
batch_sample, batch_mask = next(loader)
batch_sample, batch_mask = batch_sample.to(device), batch_mask.to(device)
batch_output = model(batch_sample)
batch_predictions = batch_output.argmax(dim=1)
visualise_batch_predictions(batch_sample, batch_mask.unsqueeze(1), batch_predictions.unsqueeze(1), rescale=True, bands=bands)